# Installation and imports

In [1]:
!pip install kfmd --upgrade --user
!pip install pandas --upgrade --user

from kfmd import metadata
import pandas
from datetime import datetime


  Created wheel for kfmd: filename=kfmd-0.1.8-py3-none-any.whl size=65103 sha256=fa3430194bc93f2af0545ea85b8ad22b408381593b3944c01bff6cf4309cac73
  Stored in directory: /home/jovyan/.cache/pip/wheels/3c/b9/0a/844e5a780190cb6da58eef3d6116e8ee1fca2030da6f7d3489
Successfully built kfmd
Requirement already up-to-date: pandas in ./.local/lib/python3.7/site-packages (1.0.3)


Create a workspace, run and execution

In [2]:
ws1 = metadata.Workspace(
    # Connect to metadata-service in namespace kubeflow in k8s cluster.
    backend_url_prefix="metadata-service.kubeflow.svc.cluster.local:8080",
    name="ws1",
    description="a workspace for testing",
    labels={"n1": "v1"})
r = metadata.Run(
    workspace=ws1,
    name="run-" + datetime.utcnow().isoformat("T") ,
    description="a run in ws_1",
)
exec = metadata.Execution(
    name = "execution" + datetime.utcnow().isoformat("T") ,
    workspace=ws1,
    run=r,
    description="execution example",
)

Log data set, model and its evaluation

In [3]:
data_set = exec.log_input(
        metadata.DataSet(
            description="an example data",
            name="mytable-dump",
            owner="owner@my-company.org",
            uri="file://path/to/dataset",
            version="v1.0.0",
            query="SELECT * FROM mytable"))
model = exec.log_output(
    metadata.Model(
            name="MNIST",
            description="model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist",
            model_type="neural network",
            training_framework={
                "name": "tensorflow",
                "version": "v1.0"
            },
            hyperparameters={
                "learning_rate": 0.5,
                "layers": [10, 3, 1],
                "early_stop": True
            },
            version="v0.0.1",
            labels={"mylabel": "l1"}))
metrics = exec.log_output(
    metadata.Metrics(
            name="MNIST-evaluation",
            description="validating the MNIST model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist-eval.csv",
            data_set_id=data_set.id,
            model_id=model.id,
            metrics_type=metadata.Metrics.VALIDATION,
            values={"accuracy": 0.95},
            labels={"mylabel": "l1"}))

List all the models in the workspace

In [4]:
pandas.DataFrame.from_dict(ws1.list(metadata.Model.ARTIFACT_TYPE_NAME))

,id,workspace,run,create_time,description,model_type,name,owner,version,uri,training_framework,hyperparameters,labels
0,5,ws1,run-2020-04-12T04:23:42.752362,2020-04-12T04:23:47.473236Z,model to recognize handwritten digits,neural network,MNIST,someone@kubeflow.org,v0.0.1,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'}


Get basic lineage

In [5]:
print("model id is %s\n" % model.id)

model id is 5



Find the execution that produces this model.

In [6]:
output_events = ws1.client.list_events2(model.id).events
assert len(output_events) == 1
execution_id = output_events[0].execution_id
print(execution_id)

3


Find all events related to that execution.

In [7]:
all_events = ws1.client.list_events(execution_id).events
assert len(all_events) == 3
print("\nAll events related to this model:")
pandas.DataFrame.from_dict([e.to_dict() for e in all_events])


All events related to this model:


,artifact_id,execution_id,path,type,milliseconds_since_epoch
0,4,3,None,INPUT,1586665427461
1,5,3,None,OUTPUT,1586665427498
2,6,3,None,OUTPUT,1586665427555
